In [1]:
import pycap
print pycap.__file__
print pycap.__git_branch__,pycap.__git_commit_hash__
from pycap import initialize_data,report_data,plot_data
import numpy
from matplotlib import pyplot
%matplotlib inline

/Users/qdi/Projects/cap/clang-3.7.0/install/opt/cap/python/pycap/__init__.pyc
master 6b47eb4bf513207b244272a4e1256d26a9544ae2


In [2]:
from pycap import TimeEvolution,EndCriterion

In [3]:
class Stage:
    def __init__(self,ptree):
        self.evolve_one_time_step=TimeEvolution.factory(ptree)
        self.end_criterion=EndCriterion.factory(ptree)
        self.time_step=ptree.get_double('time_step')
    def run(self,device,data={}):
        if not data:
            time=0.0
        elif len(data['time'])>0:
            time=data['time'][-1]
        else:
            time=0.0
        self.end_criterion.reset(time,device)
        while not self.end_criterion.check(time,device):
            time+=self.time_step
            self.evolve_one_time_step(device,self.time_step)
            if data:
                report_data(data,time,device)

class MultiStage:
    def __init__(self,ptree):
        self.stages=[]
        for stage in range(ptree.get_int('stages')):
            child=ptree.get_child('stage_'+str(stage))
            try:
                child.get_double('time_step')
            except:
                time_step=ptree.get_double('time_step')
                child.put_double('time_step',time_step)
            self.stages.append(Stage(child))
        self.cycles=ptree.get_int('cycles')
    def run(self,device,data={}):
        for cycle in range(self.cycles):
            for stage in self.stages:
                stage.run(device,data)

In [4]:
input_database=pycap.PropertyTree()
input_database.parse_xml('multi_stage.xml')
device=pycap.EnergyStorageDevice(input_database)

data=initialize_data()
experiment=MultiStage(input_database.get_child('operating_conditions'))
experiment.run(device,data)


TypeError: 'NoneType' object is not callable

In [ ]:
plot_data(data)

In [ ]:
class TimeEvolution:
    def factory(ptree):
        mode=ptree.get_string('type')
    
        if   mode=='constant_voltage' or mode=='potentiostatic':
            constant_voltage=ptree.get_double('voltage')
            def evolve_one_time_step_constant_voltage(device,time_step):
                device.evolve_one_time_step_constant_voltage(time_step,constant_voltage)
            return evolve_one_time_step_constant_voltage

        elif mode=='constant_current' or mode=='galvanostatic':
            constant_current=ptree.get_double('current')
            def evolve_one_time_step_constant_current(device,time_step):
                device.evolve_one_time_step_constant_current(time_step,constant_current)
            return evolve_one_time_step_constant_current

        elif mode=='constant_power'  :
            constant_power  =ptree.get_double('power'  )
            def evolve_one_time_step_constant_power  (device,time_step):
                device.evolve_one_time_step_constant_power  (time_step,constant_power  )
            return evolve_one_time_step_constant_power

        elif mode=='constant_load'   :
            constant_load   =ptree.get_double('load'   )
            def evolve_one_time_step_constant_load   (device,time_step):
                device.evolve_one_time_step_constant_load   (time_step,constant_load   )
            return evolve_one_time_step_constant_load
        
    factory=staticmethod(factory)

In [ ]:
import operator

class EndCriterion:
    def check(self,time,device):
        raise NotImplementedError
    def reset(self,time,device):
        raise NotImplementedError
    def factory(ptree):
        type=ptree.get_string('end_criterion')
        if   type=='time':
            return TimeLimit(ptree)
        elif type=='voltage_greater_than':
            return VoltageLimit(ptree,operator.ge)
        elif type=='voltage_less_than'   :
            return VoltageLimit(ptree,operator.le)
        elif type=='current_greater_than':
            return CurrentLimit(ptree,operator.ge)
        elif type=='current_less_than'   :
            return CurrentLimit(ptree,operator.le)
        else:
            raise NameError("invalid EndCriterion type '"+type+"'")

    factory=staticmethod(factory)

class TimeLimit(EndCriterion):
    def __init__(self,ptree):
        self.duration=ptree.get_double('duration')
    def check(self,time,device):
        return time-self.tick>=self.duration
    def reset(self,time,device):
        self.tick=time

class VoltageLimit(EndCriterion):
    def __init__(self,ptree,compare):
        self.voltage_limit=ptree.get_double('voltage_limit')
        self.compare=compare
    def check(self,time,device):
        return not self.compare(device.get_voltage(),self.voltage_limit)
    def reset(self,time,device):
        pass

class CurrentLimit(EndCriterion):
    def __init__(self,ptree,compare):
        self.current_limit=ptree.get_double('current_limit')
        self.compare=compare
    def check(self,time,device):
        return not self.compare(device.get_current(),self.current_limit)
    def reset(self,time,device):
        pass